# **Decision Trees**

The Wisconsin Breast Cancer Dataset(WBCD) can be found here(https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data)

This dataset describes the characteristics of the cell nuclei of various patients with and without breast cancer. The task is to classify a decision tree to predict if a patient has a benign or a malignant tumour based on these features.

Attribute Information:
```
#  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
```

In [1]:
import pandas as pd
headers = ["ID","CT","UCSize","UCShape","MA","SECSize","BN","BC","NN","Mitoses","Diagnosis"]
data = pd.read_csv('breast-cancer-wisconsin.data', na_values='?',    
         header=None, index_col=['ID'], names = headers) 
data = data.reset_index(drop=True)
data = data.fillna(0)
data.describe()

,CT,UCSize,UCShape,MA,SECSize,BN,BC,NN,Mitoses,Diagnosis
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.463519,3.437768,2.866953,1.589413,2.689557
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.640708,2.438364,3.053634,1.715078,0.951273
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


1. a) Implement a decision tree (you can use decision tree implementation from existing libraries).

### Note that the accuracy of the classifiers change from run to run and after multiple iterations I have tried report average accuracies.

In [3]:
print(f"Shape of data: {data.shape}")

Shape of data: (699, 10)


In [2]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

X = data.iloc[:,:9] # features
y = data.iloc[:,9]  # class label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 1/9, train_size = 8/9)

In [5]:
model = DecisionTreeClassifier() # create the decision tree model object
model = model.fit(X_train,y_train) # train the decision tree
y_pred = model.predict(X_test) # predict class labels for test data
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)) # model accuracy

Accuracy: 0.9428571428571428


1. b) Train a decision tree object of the above class on the WBC dataset using misclassification rate, entropy and Gini as the splitting metrics.

In [6]:
gini_accuracy_sum = 0
entropy_accuracy_sum = 0

# Average over 100 runs
for i in range(100):
    
    model = DecisionTreeClassifier(criterion="gini")
    model = model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    gini_accuracy_sum = gini_accuracy_sum + metrics.accuracy_score(y_test, y_pred)
    
    model = DecisionTreeClassifier(criterion="entropy")
    model = model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    entropy_accuracy_sum = entropy_accuracy_sum + metrics.accuracy_score(y_test, y_pred)
    
print("Average over 100 runs")
print("Gini Accuracy:",gini_accuracy_sum/100)
print("Entropy Accuracy:",entropy_accuracy_sum/100)

Average over 100 runs
Gini Accuracy: 0.9318571428571419
Entropy Accuracy: 0.9377142857142854


1. c) Report the accuracies in each of the above splitting metrics and give the best result. 

We find that:
* Gini Accuracy: 0.9318571428571419
* Entropy Accuracy: 0.9377142857142854

Both criterion give similar results, with entropy being slightly better in most of the runs.  
Avg accuracy is about 93 to 94% and best result is never more than 95%.

1. d) Experiment with different approaches to decide when to terminate the tree (number of layers, purity measure, etc). Report and give explanations for all approaches. 

In [7]:
import numpy as np
import matplotlib.pyplot as plt

################# Hyper-parameters #####################
max_depth = [5, 6, 7, 8, 9, 10, 11, 12]
min_impurity_decrease = np.linspace(0,1,40)
criterion = ['gini', 'entropy']
########################################################

max_depth_parameter = None
min_impurity_decrease_parameter = None
criterion_parameter = None

highest_acc = -np.inf

for mx_depth in max_depth:
    for mn_impurity_decrease in min_impurity_decrease:
        for criteria in criterion:
                    
                    accuracy_sum = 0
                    
                    for i in range(5):
                        model = DecisionTreeClassifier(criterion=criteria, max_depth=mx_depth, min_impurity_decrease=mn_impurity_decrease)
                        model = model.fit(X_train,y_train)
                        y_pred = model.predict(X_val)
                        accuracy_sum = accuracy_sum + metrics.accuracy_score(y_val, y_pred)
                    
                    accuracy_sum = accuracy_sum / 5
                    
                    if highest_acc < accuracy_sum:
                        highest_acc = accuracy_sum
                        max_depth_parameter = mx_depth
                        min_impurity_decrease_parameter = mn_impurity_decrease
                        criterion_parameter = criteria

                        
model = DecisionTreeClassifier(criterion=criterion_parameter, max_depth=max_depth_parameter, min_impurity_decrease=min_impurity_decrease_parameter)
model = model.fit(X_train,y_train)
y_pred = model.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred)

print(f"Accuracy after parameter tuning = {acc}")
print(f"Parameters for tuned decision tree are:")
print(f"• max_depth = {max_depth_parameter}")
print(f"• min_impurity_decrease = {min_impurity_decrease_parameter}")
print(f"• criterion = {criterion_parameter}")

Accuracy after parameter tuning = 0.9428571428571428
Parameters for tuned decision tree are:
• max_depth = 9
• min_impurity_decrease = 0.0
• criterion = gini


To tune the decision tree, I have generated multiple possible combinations for 3 of the features. For each set, I have trained 5 decision trees and calculated the average accuracy on validation set. After this I have reported the set of parameters giving highest average accuracy on validation set and reported its accuracy on test set.  

The parameters obtained are usually similar to `max_depth = 8`, `min_impurity_decrease = 0.0` and `criterion = entropy`. Maximum depth is usually near 6 to 8. This because as the dataset is not very large, by having more depth we will overfit. Minimum impurity decrease is mostly 0 which tells us about the data. We need to make some cuts with no information gain to be able to classify later. Gini and entropy are both common but gini is slightly less frequent than entropy as it is fast but does less computations than entropy.

2. What is boosting, bagging and  stacking?
Which class does random forests belong to and why?

Answer:  

* **Bagging** (Bootstrap Aggregating) is used to reduce variance of our prediction by generating training data using the original dataset using combinations with repetitions to produce multiple sets of the same cardinality. Now each of these sets is used to train a model and majority vote is taken to decide the class.

* **Boosting**: In this approach we first use subsets of the original data to produce a series of averagely performing models and then boosts their performance by combining them together using a particular cost function. In boosting the subset creation is not random and depends upon the performance of the previous models. Every new subsets contains the elements that were misclassified by previous models.

* **Stacking** is a similar to boosting with several different models are used on the data. We estimate the target using outputs of every model.

Random forest belongs to bagging as we have multiple trees and our final prediction is based on the prediction of all the trees in the forest.

3. Implement random forest algorithm using different decision trees . 

In [8]:
from collections import Counter
import numpy as np

def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

def RandomForest(n_trees):
    trees = []
    for _ in range(n_trees):
        tree = DecisionTreeClassifier(
            max_depth = max_depth_parameter,
            min_impurity_decrease = min_impurity_decrease_parameter,
            criterion = criterion_parameter
        )
        tree.fit(X_train, y_train)
        trees.append(tree)
        
    tree_preds = np.array([tree.predict(X_val) for tree in trees])
    tree_preds = np.swapaxes(tree_preds, 0, 1)
    y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
    acc = metrics.accuracy_score(y_pred, y_val)
    return acc

for i in [13, 24, 31, 39, 40, 45, 50, 59, 63, 78, 86, 91]:
    print(f"NUMBER OF TREES = {i}    ACCURACY = {RandomForest(i)}")

NUMBER OF TREES = 13    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 24    ACCURACY = 0.9857142857142858
NUMBER OF TREES = 31    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 39    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 40    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 45    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 50    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 59    ACCURACY = 0.9857142857142858
NUMBER OF TREES = 63    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 78    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 86    ACCURACY = 0.9714285714285714
NUMBER OF TREES = 91    ACCURACY = 0.9714285714285714


4. Report the accuracies obtained after using the Random forest algorithm and compare it with the best accuracies obtained with the decision trees. 

Answer  
In majority of the runs, best perforamce of random forest is better than better performace of single decision tree. Accuracy for both have been printed and can be compared.

5. Submit your solution as a separate pdf in the final zip file of your submission


Compute a decision tree with the goal to predict the food review based on its smell, taste and portion size.

(a) Compute the entropy of each rule in the first stage.

(b) Show the final decision tree. Clearly draw it.

Submit a handwritten response. Clearly show all the steps.

